In [ ]:
!pip install --upgrade --no-cache-dir gdown
import gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
# download src folder
url = "https://drive.google.com/drive/folders/1fhj09p3ByE7RgdiUUPUhkuKVLN5h3qvd"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/src/dataloader.py',
 '/content/src/losses.py',
 '/content/src/model.py',
 '/content/src/visualization.py']

In [ ]:
from PIL import Image
import numpy as np
import os
import argparse
import glob

In [ ]:
from numpy import asarray,savez_compressed
from keras_preprocessing.image import img_to_array
from keras_preprocessing.image import load_img

In [ ]:
import tensorflow as tf
from keras.layers import Layer, InputSpec, Reshape, Activation, Conv2D, Conv2DTranspose, SeparableConv2D, Dropout
from keras.layers import Input, Add, Concatenate, Lambda,LeakyReLU,AveragePooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.models import Model
from keras.initializers import RandomNormal
from functools import partial
from src.losses import *

class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        if type(padding) == int:
            padding = (padding, padding)
        self.padding = padding
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "padding": self.padding
        })
        return config

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')
def novel_residual_block(X_input, filters,base):

    name_base = base + '/branch'
    X = X_input
    X = ReflectionPadding2D((1,1),name=name_base + '1/rf')(X)
    X = SeparableConv2D(filters, kernel_size=(3,3), strides=(1,1),dilation_rate=1, padding='valid',name=name_base + '1/sepconv')(X)
    X = BatchNormalization(axis=3, center=True, scale=True, name=name_base + '1/BNorm')(X)
    X = LeakyReLU(alpha=0.2,name=name_base + '1/LeakyRelu')(X)

    ## Branch 1 ext1
    X_branch_1 = ReflectionPadding2D((1,1),name=name_base + '1_1/rf')(X)
    X_branch_1 = SeparableConv2D(filters, kernel_size=(3,3), strides=(1,1), padding='valid',name=name_base + '1_1/sepconv')(X_branch_1)
    X_branch_1 = BatchNormalization(axis=3, center=True, scale=True, name=name_base + '1_1/BNorm')(X_branch_1)
    X_branch_1 = LeakyReLU(alpha=0.2,name=name_base + '1_1/LeakyRelu')(X_branch_1)

    ## Branch 2
    X_branch_2 = ReflectionPadding2D((2,2),name=name_base + '2/rf')(X)
    X_branch_2 = SeparableConv2D(filters, kernel_size=(3,3), strides=(1,1), dilation_rate=2, padding='valid',name=name_base + '2/sepconv')(X_branch_2)
    X_branch_2 = BatchNormalization(axis=3, center=True, scale=True, name=name_base + '2/BNorm')(X_branch_2)
    X_branch_2 = LeakyReLU(alpha=0.2,name=name_base + '2/LeakyRelu')(X_branch_2)
    X_add_branch_1_2 = Add(name=name_base + '1/add_branch1_2')([X_branch_2,X_branch_1])
    X = Add(name=name_base + '1/add_skip')([X_input, X_add_branch_1_2])
    return X

def disc_res_block(X_input, filters,base):

    name_base = base + '/branch'
    X = X_input

    ## Branch 1
    X_branch_1 = ReflectionPadding2D((1,1),name=name_base + '1_1/rf')(X)
    X_branch_1 = Conv2D(filters, kernel_size=(2,2), strides=(1,1), dilation_rate=2, padding='valid',kernel_initializer=RandomNormal(stddev=0.02),name=name_base + '1_1/conv')(X_branch_1)
    X_branch_1 = BatchNormalization(axis=3, center=True, scale=True, name=name_base + '1_1/BNorm')(X_branch_1)
    X_branch_1 = LeakyReLU(alpha=0.2,name=name_base + '1_1/LeakyRelu')(X_branch_1)

    ## Branch 2
    X_branch_2 = ReflectionPadding2D((1,1),name=name_base + '1_2/rf')(X)
    X_branch_2 = SeparableConv2D(filters, kernel_size=(2,2), strides=(1,1), dilation_rate=2, padding='valid',kernel_initializer=RandomNormal(stddev=0.02),name=name_base + '1_2/sepconv')(X_branch_2)
    X_branch_2 = BatchNormalization(axis=3, center=True, scale=True, name=name_base + '1_2/BNorm')(X_branch_2)
    X_branch_2 = LeakyReLU(alpha=0.2,name=name_base + '1_2/LeakyRelu')(X_branch_2)
    X_add_branch_1_2 = Add(name=name_base + '1_1/add_branch1_2')([X_branch_2,X_branch_1])

    X = Add(name=name_base + '2/add_skip')([X_input, X_add_branch_1_2])
    return X
    
def SFA(X,filters,i):
    X_input = X
    X = SeparableConv2D(filters, kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer=RandomNormal(stddev=0.02),name="SFA_"+str(i+1)+"/conv1")(X)
    X = BatchNormalization(name="Attention_"+str(i+1)+"/BNorm1")(X)
    X = LeakyReLU(alpha=0.2,name="Attention_"+str(i+1)+"/leakyReLU1")(X)
    X = Add(name="Attention_"+str(i+1)+"/add1")([X_input,X])

    X = Conv2D(filters, kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer=RandomNormal(stddev=0.02),name="SFA_"+str(i+1)+"/conv2")(X)
    X = BatchNormalization(name="Attention_"+str(i+1)+"/BNorm2")(X)
    X = LeakyReLU(alpha=0.2,name="Attention_"+str(i+1)+"/leakyReLU2")(X)

    X = Add(name="Attention_"+str(i+1)+"/add2")([X_input,X])
    return X
def encoder_block(X,down_filters,i):
    X = Conv2D(down_filters, kernel_size=(4,4), strides=(2,2), padding='same',kernel_initializer=RandomNormal(stddev=0.02),name="down_conv_"+str(i+1))(X)
    X = BatchNormalization(name="down_bn_"+str(i+1))(X)
    X = LeakyReLU(alpha=0.2,name="down_leakyRelu_"+str(i+1))(X)
    return X

def decoder_block(X,up_filters,i):
    X = Conv2DTranspose(filters=up_filters, kernel_size=(4,4), strides=(2,2), padding='same',kernel_initializer=RandomNormal(stddev=0.02),name="up_convtranpose_"+str(i+1) )(X)
    X = BatchNormalization(name="up_bn_"+str(i+1))(X)
    X = LeakyReLU(alpha=0.2,name="up_leakyRelu_"+str(i+1))(X)
    return X

def coarse_generator(img_shape=(256, 256, 3),mask_shape=(256,256,1),ncf=64, n_downsampling=2, n_blocks=9, n_channels=1):
    X_input = Input(img_shape,name="input")
    X_mask  = Input(mask_shape,name="input_mask")
    X = Concatenate(axis=-1,name="concat")([X_input,X_mask])
    X = ReflectionPadding2D((3,3))(X)
    X = Conv2D(ncf, kernel_size=(7,7), strides=(1,1), padding='valid',kernel_initializer=RandomNormal(stddev=0.02),name="conv1")(X)
    X = BatchNormalization(name="bn_1")(X)
    X_pre_down = LeakyReLU(alpha=0.2,name="leakyRelu_1")(X)

    # Downsampling layers
    down_filters = ncf * pow(2,0) * 2
    X_down1 = encoder_block(X,down_filters,0)
    down_filters = ncf * pow(2,1) * 2
    X_down2 = encoder_block(X_down1,down_filters,1)
    X = X_down2


    # Novel Residual Blocks
    res_filters = pow(2,n_downsampling)
    for i in range(n_blocks):
        X = novel_residual_block(X, ncf*res_filters,base="block_"+str(i+1))


    # Upsampling layers
    up_filters  =int(ncf * pow(2,(n_downsampling - 0)) / 2) 
    X_up1 = decoder_block(X,up_filters,0)
    X_up1_att = SFA(X_down1,ncf*2,0)
    X_up1_add = Add(name="skip_1")([X_up1_att,X_up1])
    up_filters  =int(ncf * pow(2,(n_downsampling - 1)) / 2) 
    X_up2 = decoder_block(X_up1_add,up_filters,1)
    X_up2_att = SFA(X_pre_down,ncf,1)
    X_up2_add = Add(name="skip_2")([X_up2_att,X_up2])
    feature_out = X_up2_add
    print("X_feature",feature_out.shape)
    X = ReflectionPadding2D((3,3),name="final/rf")(X_up2_add)
    X = Conv2D(n_channels, kernel_size=(7,7), strides=(1,1), padding='valid',kernel_initializer=RandomNormal(stddev=0.02),name="final/conv")(X)
    X = Activation('tanh',name="tanh")(X)


    model = Model(inputs=[X_input,X_mask], outputs=[X,feature_out],name='G_Coarse')
    model.compile(loss=['mse',None], optimizer=Adam(lr=0.0002, beta_1=0.5, beta_2=0.999))

    model.summary()
    return model

def fine_generator(x_coarse_shape=(256,256,64),input_shape=(512, 512, 3), mask_shape=(512,512,1), nff=64, n_blocks=3, n_coarse_gen=1,n_channels = 1):

    
    X_input = Input(shape=input_shape,name="input")
    X_mask = Input(shape=mask_shape,name="input_mask")
    X_coarse = Input(shape=x_coarse_shape,name="x_input")
    print("X_coarse",X_coarse.shape)
    for i in range(1, n_coarse_gen+1):
        
        
        # Downsampling layers
        down_filters = nff * (2**(n_coarse_gen-i))
        X = Concatenate(axis=-1,name="concat")([X_input, X_mask])
        X = ReflectionPadding2D((3,3),name="rf_"+str(i))(X)
        X = Conv2D(down_filters, kernel_size=(7,7), strides=(1,1), padding='valid',kernel_initializer=RandomNormal(stddev=0.02),name="conv_"+str(i))(X)
        X = BatchNormalization(name="in_"+str(i))(X)
        X_pre_down = LeakyReLU(alpha=0.2,name="leakyRelu_"+str(i))(X)


        X_down1 = encoder_block(X,down_filters,i-1)
        # Connection from Coarse Generator
        X = Add(name="add_X_coarse")([X_coarse,X_down1])

        X = SeparableConv2D(down_filters*2, kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer=RandomNormal(stddev=0.02),name="sepconv_"+str(i))(X)
        X = BatchNormalization(name="sep_in_"+str(i))(X)
        X = LeakyReLU(alpha=0.2,name="sep_leakyRelu_"+str(i))(X)
        for j in range(n_blocks-1):
            res_filters = nff * (2**(n_coarse_gen-i)) * 2
            X = novel_residual_block(X, res_filters,base="block_"+str(j+1))

        # Upsampling layers
        up_filters = nff * (2**(n_coarse_gen-i))
        X_up1 = decoder_block(X,up_filters,i-1)
        X_up1_att = SFA(X_pre_down,up_filters,i-1)
        X_up1_add = Add(name="skip_"+str(i))([X_up1_att,X_up1])

    X = ReflectionPadding2D((3,3),name="final/rf")(X_up1_add)
    X = Conv2D(n_channels, kernel_size=(7,7), strides=(1,1), padding='valid',name="final/conv")(X)
    X = Activation('tanh',name="tanh")(X)

    model = Model(inputs=[X_input,X_mask,X_coarse], outputs=X, name='G_Fine')
    model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5, beta_2=0.999))

    model.summary()
    return model


def discriminator_ae(input_shape_fundus=(512, 512, 3),
                        input_shape_label=(512, 512, 1),
                        ndf=32, n_layers=3, activation='tanh',
                        name='Discriminator'):
    X_input_fundus = Input(shape=input_shape_fundus,name="input_fundus")
    X_input_label = Input(shape=input_shape_label,name="input_label")

    features =[]
    X = Concatenate(axis=-1,name="concat")([X_input_fundus, X_input_label])
    
    X_down1 = encoder_block(X,ndf,0)
    X_down1_res = disc_res_block(X_down1, ndf,base="block1_down")
    features.append(X_down1_res)

    down_filters = ndf
    X_down2 = encoder_block(X_down1_res,down_filters,1)
    X_down2_res = disc_res_block(X_down2,down_filters,base="block2_down")
    features.append(X_down2_res)
    
    down_filters = ndf
    X_down3 = encoder_block(X_down2_res,down_filters,2)
    X_down3_res = disc_res_block(X_down3,down_filters,base="block3_down")
    features.append(X_down3_res)
    
    up_filters = ndf
    X_up1 = decoder_block(X_down3_res,up_filters,0)
    features.append(X_up1)

    X_up2 = decoder_block(X_up1,ndf,1)
    features.append(X_up2)

    X_up3 = decoder_block(X_up2,ndf,2)
    features.append(X_up3)


    X = Conv2D(1, kernel_size=(4,4), strides=(1,1), padding='same',kernel_initializer=RandomNormal(stddev=0.02))(X_up3)
    X = Activation(activation)(X)

    model = Model(inputs=[X_input_fundus, X_input_label], outputs=[X]+features, name=name)
    model.summary()
    model.compile(loss=['mse',None,None,None,None,None,None], optimizer=Adam(lr=0.0002, beta_1=0.5, beta_2=0.999))
    return model

def RVgan(g_model_fine,g_model_coarse, d_model1, d_model2,image_shape_fine,image_shape_coarse,image_shape_x_coarse,mask_shape_fine,mask_shape_coarse,label_shape_fine,label_shape_coarse,inner_weight):
    # Discriminator NOT trainable
    d_model1.trainable = False
    d_model2.trainable = False

    in_fine= Input(shape=image_shape_fine)
    in_coarse = Input(shape=image_shape_coarse)
    in_x_coarse = Input(shape=image_shape_x_coarse)
    in_fine_mask = Input(shape=mask_shape_fine)
    in_coarse_mask = Input(shape=mask_shape_coarse)
    label_fine = Input(shape=label_shape_fine)
    label_coarse = Input(shape=label_shape_coarse)

    # Generators
    gen_out_coarse, _ = g_model_coarse([in_coarse,in_coarse_mask])
    gen_out_fine = g_model_fine([in_fine,in_fine_mask,in_x_coarse])

    # Discriminators Fine
    dis_out_1_fake = d_model1([in_fine, gen_out_fine])

    # Discriminators Coarse
    dis_out_2_fake = d_model2([in_coarse, gen_out_coarse])

    #feature matching loss
    fm1 = partial(weighted_feature_matching_loss, image_input=in_fine,real_samples=label_fine, D=d_model1, inner_weight=inner_weight)
    #fm1 = partial(feature_matching_loss, image_input=in_fine,real_samples=label_fine, D=d_model1)
    fm2 = partial(weighted_feature_matching_loss, image_input=in_coarse,real_samples=label_coarse, D=d_model2, inner_weight=inner_weight)
    #fm2 = partial(feature_matching_loss, image_input=in_coarse,real_samples=label_coarse, D=d_model2)

    model = Model([in_fine,in_coarse,in_x_coarse,in_fine_mask,in_coarse_mask,label_fine,label_coarse], [dis_out_1_fake[0],
                                                    dis_out_2_fake[0],
                                                    gen_out_fine,
                                                    gen_out_coarse,
                                                    gen_out_coarse,
                                                    gen_out_fine,
                                                    gen_out_coarse,
                                                    gen_out_fine
                                                    ])

    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['hinge', 
                    'hinge',
                    fm1,
                    fm2,
                    'hinge',
                    'hinge',
                    'mse',
                    'mse'
                    ], 
              optimizer=opt,loss_weights=[1,1,
                                          1,
                                          1,
                                          10,10,10,10
                                          ])
    model.summary()
    return model

In [ ]:
#from src.model import coarse_generator,fine_generator,RVgan,discriminator_ae
from src.visualization import summarize_performance, summarize_performance_global, plot_history, to_csv
from src.dataloader import resize, generate_fake_data_coarse, generate_fake_data_fine, generate_real_data, generate_real_data_random, load_real_data
import argparse
import time
import os
from numpy import load
import gc
import keras.backend as K

## Training

In [ ]:
# download NPZ file
url = "https://drive.google.com/drive/folders/1AT0Yz4n0IRyNIDHyf-FRoh-dpLwRLHqs"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/NPZ/MIC.npz']

In [ ]:
epochs = 16
batch_size = 16
npz_file = 'NPZ/MIC.npz' # help='path/to/npz/file' ## CHANGED (NPZ/CHASE.npz)
input_dim = 128
savedir = 'ICGAN' # help='path/to/save_directory'
resume_training = 'no' # choices=['yes','no'])
weight_name_global = None # help='path/to/global/weight/.h5 file'
weight_name_local = None # help='path/to/local/weight/.h5 file']
inner_weight = 0.5

K.clear_session()
gc.collect()
start_time = time.time()
dataset = load_real_data(npz_file)
print('Loaded', dataset[0].shape, dataset[1].shape)

# define input shape based on the loaded dataset
in_size = input_dim
image_shape_coarse = (in_size//2,in_size//2,3)
mask_shape_coarse = (in_size//2,in_size//2,1)
label_shape_coarse = (in_size//2,in_size//2,1)


image_shape_fine = (in_size,in_size,3)
mask_shape_fine = (in_size,in_size,1)
label_shape_fine = (in_size,in_size,1)

image_shape_xglobal = (in_size//2,in_size//2,128)
ndf=64
ncf=128
nff=128

d_model1 = discriminator_ae(image_shape_fine,label_shape_fine,ndf,name="D1") 
d_model2 = discriminator_ae(image_shape_coarse,label_shape_coarse,ndf,name="D2")


g_model_fine = fine_generator(x_coarse_shape=image_shape_xglobal,input_shape=image_shape_fine,mask_shape=mask_shape_fine,nff=nff,n_blocks=3)
g_model_coarse = coarse_generator(img_shape=image_shape_coarse,mask_shape=mask_shape_coarse,n_downsampling=2, n_blocks=9, ncf=ncf,n_channels=1)


if resume_training =='yes':
  #weight_name_global = "global_model_000070.h5"
  g_model_coarse.load_weights(weight_name_global)

  #weight_name_local = "local_model_000070.h5"
  g_model_fine.load_weights(weight_name_local)
  
rvgan_model = RVgan(g_model_fine,g_model_coarse, d_model1, d_model2,
              image_shape_fine,image_shape_coarse,image_shape_xglobal,mask_shape_fine,mask_shape_coarse,label_shape_fine,label_shape_coarse, inner_weight)

Loaded (836, 128, 128, 3) (836, 128, 128, 1)
Model: "D1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_fundus (InputLayer)      [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_label (InputLayer)       [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 concat (Concatenate)           (None, 128, 128, 4)  0           ['input_fundus[0][0]',           
                                                    

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "D2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_fundus (InputLayer)      [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 input_label (InputLayer)       [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 concat (Concatenate)           (None, 64, 64, 4)    0           ['input_fundus[0][0]',           
                                                                  'input_label[0][0]']            
                                                                                                  
 down_conv_1 (Conv2D)           (None, 32, 32, 64)   4160        ['concat[0][0]']                

In [ ]:
def train(d_model1, d_model2, g_global_model, g_local_model, 
          gan_model, dataset, n_epochs=20, n_batch=1, n_patch=[64,32],savedir='ICGAN'):
    
    if not os.path.exists(savedir):
      os.makedirs(savedir)
    # unpack dataset
    trainA, _, _ = dataset
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    # lists for storing loss, for plotting later
    d1_hist, d2_hist, d3_hist, d4_hist =  list(),list(), list(), list()
    fm1_hist,fm2_hist = list(),list()
    g_global_hist, g_local_hist, gan_hist =  list(), list(), list()
    g_global_recon_hist, g_local_recon_hist =list(),list()
    # manually enumerate epochs
    b = 0
    start_time = time.time()
    for k in range(n_epochs):
        for i in range(bat_per_epo):
          d_model1.trainable = True
          d_model2.trainable = True
          gan_model.trainable = False
          g_global_model.trainable = False
          g_local_model.trainable = False
          for j in range(2):
              # select a batch of real samples 
              [X_realA, X_realB, X_realC], [y1,y2] = generate_real_data(dataset, i, n_batch, n_patch)

              
              # generate a batch of fake samples for Coarse Generator
              out_shape = (int(X_realA.shape[1]/2),int(X_realA.shape[2]/2))
              [X_realA_half,X_realB_half, X_realC_half] = resize(X_realA,X_realB,X_realC,out_shape)
              [X_fakeC_half, x_global], y1_coarse = generate_fake_data_coarse(g_global_model, X_realA_half, X_realB_half, n_patch)


              # generate a batch of fake samples for Fine Generator
              X_fakeC, y1_fine= generate_fake_data_fine(g_local_model, X_realA, X_realB, x_global, n_patch)


              ## FINE DISCRIMINATOR  
              # update discriminator for real samples
              d_loss1 = d_model1.train_on_batch([X_realA, X_realC], y1)[0]
              # update discriminator for generated samples
              d_loss2 = d_model1.train_on_batch([X_realA, X_fakeC], y1_fine)[0]

              #d_loss1 = 0.5*(d_loss1_real[0]+d_loss1_fake[0])

              
              #d_loss2 = 0.5*(d_loss2_real[0]+d_loss2_fake[0])

              ## COARSE DISCRIMINATOR  
              # update discriminator for real samples
              d_loss3 = d_model2.train_on_batch([X_realA_half, X_realC_half], y2)[0]
              # update discriminator for generated samples
              d_loss4 = d_model2.train_on_batch([X_realA_half, X_fakeC_half], y1_coarse)[0]
          
          #if n_steps%425 ==0:

          # turn Global G1 trainable
          d_model1.trainable = False
          d_model2.trainable = False
          gan_model.trainable = False
          g_global_model.trainable = True
          g_local_model.trainable = False
          
          

          # select a batch of real samples for Local enhancer
          [X_realA, X_realB, X_realC], _ = generate_real_data(dataset, i,n_batch, n_patch)

          # Global Generator image fake and real
          out_shape = (int(X_realA.shape[1]/2),int(X_realA.shape[2]/2))
          [X_realA_half,X_realB_half, X_realC_half] = resize(X_realA,X_realB,X_realC,out_shape)
          [X_fakeC_half, x_global], _ = generate_fake_data_coarse(g_global_model, X_realA_half, X_realB_half, n_patch)
          

          # update the global generator
          g_global_loss,_ = g_global_model.train_on_batch([X_realA_half,X_realB_half], X_realC_half)

          
          d_model1.trainable = False
          d_model2.trainable = False
          gan_model.trainable = False
          g_global_model.trainable = False
          g_local_model.trainable = True
          
          # update the Local Enhancer 
          g_local_loss = g_local_model.train_on_batch([X_realA,X_realB,x_global], X_realC)
          

          # turn G1, G2 and GAN trainable, not D1,D2 and D3
          d_model1.trainable = False
          d_model2.trainable = False
          gan_model.trainable = True
          g_global_model.trainable = True
          g_local_model.trainable = True
          # update the generator
          sample_weight = [np.full((16, 128, 128, 1), 1.0), np.full((16, 64, 64, 1), 1.0)]
          gan_loss,_,_,fm1_loss,fm2_loss,_,_,g_global_recon_loss, g_local_recon_loss = gan_model.train_on_batch([X_realA,X_realA_half,x_global,X_realB,X_realB_half,X_realC,X_realC_half], 
                                                                                                                                                      [y1, y2,
                                                                                                                                                        X_fakeC,X_fakeC_half,
                                                                                                                                                        X_fakeC_half,X_fakeC,
                                                                                                                                                        X_fakeC_half,X_fakeC], sample_weight = sample_weight)

          # summarize performance
          print('>%d, d1[%.3f] d2[%.3f] d3[%.3f] d4[%.3f] fm1[%.3f] fm2[%.3f] g_g[%.3f] g_l[%.3f] g_g_r[%.3f] g_l_r[%.3f] gan[%.3f]' % 
                (i+1, d_loss1, d_loss2, d_loss3, d_loss4, 
                  fm1_loss, fm2_loss, 
                  g_global_loss, g_local_loss, 
                  g_global_recon_loss, g_local_recon_loss, gan_loss))
                                                                                                                              
          d1_hist.append(d_loss1)
          d2_hist.append(d_loss2)
          d3_hist.append(d_loss3)
          d4_hist.append(d_loss4)
          fm1_hist.append(fm1_loss)
          fm2_hist.append(fm2_loss)
          g_global_hist.append(g_global_loss)
          g_local_hist.append(g_local_loss)
          g_global_recon_hist.append(g_global_recon_loss)
          g_local_recon_hist.append(g_local_recon_loss)
          gan_hist.append(gan_loss)
          # summarize model performance
      #if (i+1) % (bat_per_epo * 1) == 0:
        summarize_performance_global(b, g_global_model, dataset, n_samples=3,savedir=savedir)
          
        summarize_performance(b, g_global_model,g_local_model, dataset, n_samples=3,savedir=savedir)
        b = b + 1
        per_epoch_time = time.time()
        total_per_epoch_time = (per_epoch_time - start_time)/3600.0
        print(total_per_epoch_time)
    plot_history(d1_hist, d2_hist, d3_hist, d4_hist, fm1_hist, fm2_hist, g_global_hist,g_local_hist, g_global_recon_hist, g_local_recon_hist, gan_hist,savedir=savedir)
    to_csv(d1_hist, d2_hist, d3_hist, d4_hist, fm1_hist, fm2_hist, g_global_hist,g_local_hist, g_global_recon_hist, g_local_recon_hist, gan_hist,savedir=savedir)

In [ ]:
train(d_model1, d_model2,g_model_coarse, g_model_fine, rvgan_model, dataset, n_epochs = epochs, n_batch = batch_size, n_patch=[128,64],savedir = savedir)

end_time = time.time()
time_taken = (end_time-start_time)/3600.0
print(time_taken)

1/1 [==============================] - 0s 24ms/step


>1, d1[1.204] d2[0.992] d3[1.738] d4[0.608] fm1[0.000] fm2[0.000] g_g[2.153] g_l[1.241] g_g_r[0.000] g_l_r[0.000] gan[2.000]
1/1 [==============================] - 0s 22ms/step
>2, d1[1.071] d2[0.958] d3[1.512] d4[0.611] fm1[0.000] fm2[0.000] g_g[0.772] g_l[0.585] g_g_r[0.000] g_l_r[0.000] gan[2.000]
1/1 [==============================] - 0s 99ms/step
>3, d1[0.939] d2[0.906] d3[1.257] d4[0.605] fm1[0.000] fm2[0.000] g_g[0.924] g_l[0.829] g_g_r[0.000] g_l_r[0.000] gan[1.999]
1/1 [==============================] - 0s 23ms/step
>4, d1[0.847] d2[0.788] d3[1.155] d4[0.658] fm1[0.000] fm2[0.000] g_g[0.376] g_l[0.380] g_g_r[0.000] g_l_r[0.000] gan[1.998]
1/1 [==============================] - 0s 22ms/step
>5, d1[0.788] d2[0.635] d3[1.103] d4[0.581] fm1[0.000] fm2[0.000] g_g[0.265] g_l[0.316] g_g_r[0.000] g_l_r[0.000] gan[1.997]
1/1 [==============================] - 0s 24ms/step
>6, d1[0.680] d2[0.547] d3[1.005] d4[0.568] fm1[0.000] fm2[0.000] g_g[0.457] g_l[0.507] g_g_r[0.000] g_l_r[0.000] g

>Saved: ICGAN/global_plot_000002.png and ICGAN/global_model_000002.h5
1/1 [==============================] - 0s 26ms/step
>Saved: ICGAN/local_plot_000002.png and ICGAN/local_model_000002.h5
0.1105421687497033
1/1 [==============================] - 0s 22ms/step
>1, d1[0.063] d2[0.169] d3[0.046] d4[0.011] fm1[0.000] fm2[0.000] g_g[1.551] g_l[1.548] g_g_r[0.000] g_l_r[0.000] gan[1.404]
1/1 [==============================] - 0s 22ms/step
>2, d1[0.032] d2[0.069] d3[0.045] d4[0.010] fm1[0.000] fm2[0.000] g_g[1.324] g_l[1.320] g_g_r[0.000] g_l_r[0.000] gan[1.427]
1/1 [==============================] - 0s 28ms/step
>3, d1[0.049] d2[0.075] d3[0.041] d4[0.010] fm1[0.000] fm2[0.000] g_g[0.789] g_l[0.794] g_g_r[0.000] g_l_r[0.000] gan[1.399]
1/1 [==============================] - 0s 23ms/step
>4, d1[0.048] d2[0.061] d3[0.046] d4[0.009] fm1[0.000] fm2[0.000] g_g[0.553] g_l[0.545] g_g_r[0.000] g_l_r[0.000] gan[1.397]
1/1 [==============================] - 0s 22ms/step
>5, d1[0.049] d2[0.063] d3[0.04

>Saved: ICGAN/global_plot_000003.png and ICGAN/global_model_000003.h5
1/1 [==============================] - 0s 23ms/step
>Saved: ICGAN/local_plot_000003.png and ICGAN/local_model_000003.h5
0.16049471682972377
1/1 [==============================] - 0s 21ms/step
>1, d1[0.005] d2[0.012] d3[0.019] d4[0.005] fm1[0.000] fm2[0.000] g_g[1.551] g_l[1.446] g_g_r[0.000] g_l_r[0.000] gan[1.265]
1/1 [==============================] - 0s 23ms/step
>2, d1[0.005] d2[0.015] d3[0.019] d4[0.004] fm1[0.000] fm2[0.000] g_g[1.324] g_l[1.244] g_g_r[0.000] g_l_r[0.000] gan[1.273]
1/1 [==============================] - 0s 22ms/step
>3, d1[0.008] d2[0.010] d3[0.017] d4[0.005] fm1[0.000] fm2[0.000] g_g[0.789] g_l[0.797] g_g_r[0.000] g_l_r[0.000] gan[1.249]
1/1 [==============================] - 0s 23ms/step
>4, d1[0.012] d2[0.011] d3[0.030] d4[0.004] fm1[0.000] fm2[0.000] g_g[0.553] g_l[0.541] g_g_r[0.000] g_l_r[0.000] gan[1.315]
1/1 [==============================] - 0s 23ms/step
>5, d1[0.018] d2[0.011] d3[0.0

>Saved: ICGAN/global_plot_000004.png and ICGAN/global_model_000004.h5
1/1 [==============================] - 0s 26ms/step
>Saved: ICGAN/local_plot_000004.png and ICGAN/local_model_000004.h5
0.21046636508570776
1/1 [==============================] - 0s 22ms/step
>1, d1[0.003] d2[0.004] d3[0.008] d4[0.002] fm1[0.000] fm2[0.000] g_g[1.551] g_l[1.543] g_g_r[0.000] g_l_r[0.000] gan[1.237]
1/1 [==============================] - 0s 23ms/step
>2, d1[0.003] d2[0.005] d3[0.007] d4[0.002] fm1[0.000] fm2[0.000] g_g[1.324] g_l[1.323] g_g_r[0.000] g_l_r[0.000] gan[1.244]
1/1 [==============================] - 0s 25ms/step
>3, d1[0.003] d2[0.004] d3[0.007] d4[0.002] fm1[0.000] fm2[0.000] g_g[0.789] g_l[0.792] g_g_r[0.000] g_l_r[0.000] gan[1.245]
1/1 [==============================] - 0s 22ms/step
>4, d1[0.005] d2[0.004] d3[0.010] d4[0.002] fm1[0.000] fm2[0.000] g_g[0.553] g_l[0.555] g_g_r[0.000] g_l_r[0.000] gan[1.301]
1/1 [==============================] - 0s 22ms/step
>5, d1[0.008] d2[0.004] d3[0.0

...

>Saved: ICGAN/global_plot_000009.png and ICGAN/global_model_000009.h5
1/1 [==============================] - 0s 21ms/step
>Saved: ICGAN/local_plot_000009.png and ICGAN/local_model_000009.h5
0.4611702990531921
1/1 [==============================] - 0s 25ms/step
>1, d1[0.001] d2[0.001] d3[0.001] d4[0.000] fm1[0.000] fm2[0.000] g_g[1.551] g_l[1.541] g_g_r[0.000] g_l_r[0.000] gan[1.311]
1/1 [==============================] - 0s 22ms/step
>2, d1[0.001] d2[0.001] d3[0.001] d4[0.000] fm1[0.000] fm2[0.000] g_g[1.324] g_l[1.317] g_g_r[0.000] g_l_r[0.000] gan[1.309]
1/1 [==============================] - 0s 22ms/step
>3, d1[0.001] d2[0.001] d3[0.001] d4[0.000] fm1[0.000] fm2[0.000] g_g[0.789] g_l[0.791] g_g_r[0.000] g_l_r[0.000] gan[1.311]
1/1 [==============================] - 0s 22ms/step
>4, d1[0.001] d2[0.001] d3[0.001] d4[0.000] fm1[0.000] fm2[0.000] g_g[0.553] g_l[0.551] g_g_r[0.000] g_l_r[0.000] gan[1.328]
1/1 [==============================] - 0s 22ms/step
>5, d1[0.002] d2[0.001] d3[0.00

>Saved: ICGAN/global_plot_000012.png and ICGAN/global_model_000012.h5
1/1 [==============================] - 0s 21ms/step
>Saved: ICGAN/local_plot_000012.png and ICGAN/local_model_000012.h5
0.6124617476595773
1/1 [==============================] - 0s 23ms/step
>1, d1[0.000] d2[0.000] d3[0.008] d4[0.004] fm1[0.000] fm2[0.000] g_g[1.551] g_l[1.537] g_g_r[0.000] g_l_r[0.000] gan[1.712]
1/1 [==============================] - 0s 22ms/step
>2, d1[0.000] d2[0.000] d3[0.007] d4[0.003] fm1[0.000] fm2[0.000] g_g[1.324] g_l[1.314] g_g_r[0.000] g_l_r[0.000] gan[1.720]
1/1 [==============================] - 0s 23ms/step
>3, d1[0.000] d2[0.000] d3[0.004] d4[0.004] fm1[0.000] fm2[0.000] g_g[0.789] g_l[0.793] g_g_r[0.000] g_l_r[0.000] gan[1.669]
1/1 [==============================] - 0s 25ms/step
>4, d1[0.001] d2[0.000] d3[0.005] d4[0.003] fm1[0.000] fm2[0.000] g_g[0.553] g_l[0.545] g_g_r[0.000] g_l_r[0.000] gan[1.702]
1/1 [==============================] - 0s 22ms/step
>5, d1[0.001] d2[0.000] d3[0.00

>Saved: ICGAN/global_plot_000014.png and ICGAN/global_model_000014.h5
1/1 [==============================] - 0s 22ms/step
>Saved: ICGAN/local_plot_000014.png and ICGAN/local_model_000014.h5
0.7130660041835573
1/1 [==============================] - 0s 23ms/step
>1, d1[0.000] d2[0.000] d3[0.003] d4[0.001] fm1[0.000] fm2[0.000] g_g[1.898] g_l[1.532] g_g_r[0.000] g_l_r[0.000] gan[1.637]
1/1 [==============================] - 0s 22ms/step
>2, d1[0.000] d2[0.000] d3[0.002] d4[0.001] fm1[0.000] fm2[0.000] g_g[1.678] g_l[1.305] g_g_r[0.000] g_l_r[0.000] gan[1.635]
1/1 [==============================] - 0s 22ms/step
>3, d1[0.000] d2[0.000] d3[0.001] d4[0.001] fm1[0.000] fm2[0.000] g_g[1.163] g_l[0.789] g_g_r[0.000] g_l_r[0.000] gan[1.544]
1/1 [==============================] - 0s 25ms/step
>4, d1[0.000] d2[0.000] d3[0.002] d4[0.001] fm1[0.000] fm2[0.000] g_g[0.972] g_l[0.543] g_g_r[0.000] g_l_r[0.000] gan[1.568]
1/1 [==============================] - 0s 23ms/step
>5, d1[0.001] d2[0.000] d3[0.00

>Saved: ICGAN/global_plot_000015.png and ICGAN/global_model_000015.h5
1/1 [==============================] - 0s 22ms/step
>Saved: ICGAN/local_plot_000015.png and ICGAN/local_model_000015.h5
0.7639010006189346
1/1 [==============================] - 0s 22ms/step
>1, d1[0.027] d2[0.324] d3[0.002] d4[0.001] fm1[0.000] fm2[0.000] g_g[1.781] g_l[1.553] g_g_r[0.000] g_l_r[0.000] gan[2.678]
1/1 [==============================] - 0s 23ms/step
>2, d1[0.049] d2[0.104] d3[0.002] d4[0.001] fm1[0.000] fm2[0.000] g_g[1.522] g_l[1.326] g_g_r[0.000] g_l_r[0.000] gan[2.707]
1/1 [==============================] - 0s 23ms/step
>3, d1[0.171] d2[0.098] d3[0.001] d4[0.001] fm1[0.000] fm2[0.000] g_g[1.035] g_l[0.792] g_g_r[0.000] g_l_r[0.000] gan[2.609]
1/1 [==============================] - 0s 25ms/step
>4, d1[0.093] d2[0.070] d3[0.001] d4[0.001] fm1[0.000] fm2[0.000] g_g[0.797] g_l[0.556] g_g_r[0.000] g_l_r[0.000] gan[2.695]
1/1 [==============================] - 0s 23ms/step
>5, d1[0.041] d2[0.106] d3[0.00

>Saved: ICGAN/global_plot_000016.png and ICGAN/global_model_000016.h5
1/1 [==============================] - 0s 23ms/step
>Saved: ICGAN/local_plot_000016.png and ICGAN/local_model_000016.h5
0.8134920516279008
Saved ICGAN/plot_line_plot_loss.png
0.8161885001262029


## Inferencec

In [ ]:
# Install tensorflow and keras

In [ ]:
# Insert MIC Dataset (test folder only)
url = "https://drive.google.com/drive/folders/1YvVXswgLX4-5aL_Gy4Nl9B8njNSiq8dq"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/MIC/test/images/ESP_016631_1770_RED.jpg',
 '/content/MIC/test/images/ESP_072116_1740_RED.jpg',
 '/content/MIC/test/images/ESP_074759_1855_RED.jpg',
 '/content/MIC/test/labels/ESP_016631_1770_RED_label.png',
 '/content/MIC/test/labels/ESP_072116_1740_RED_label.png',
 '/content/MIC/test/labels/ESP_074759_1855_RED_label.png',
 '/content/MIC/test/mask/ESP_016631_1770_RED_mask.png',
 '/content/MIC/test/mask/ESP_072116_1740_RED_mask.png',
 '/content/MIC/test/mask/ESP_074759_1855_RED_mask.png']

In [ ]:
!pip install libtiff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 4.9 MB/s 
  Created wheel for libtiff: filename=libtiff-0.4.2-cp38-cp38-linux_x86_64.whl size=284362 sha256=1c0aea8d0f7ff921534f8dec411028b2ef6b1bc507edf4823f1b57562703f27f
  Stored in directory: /root/.cache/pip/wheels/70/ef/75/99ce530b01b6a5bef81ebed8c05bd7713ca298e21b5635dbac
Successfully built libtiff


In [ ]:
import glob
import os
import time
import argparse
import numpy as np
from PIL import Image
from libtiff import TIFF
import tensorflow as tf
import cv2
import keras
from keras.optimizers import Adam
from keras.models import Model
import keras.backend as K
from src.model import coarse_generator,fine_generator


global g_local_model
global g_global_model

In [ ]:
def normalize_pred(img,mask):
    img = np.reshape(img,[1,128,128,3])
    mask = np.reshape(mask,[1,128,128,1])
    img_coarse = tf.image.resize(img, (64,64), method=tf.image.ResizeMethod.LANCZOS3)
    img_coarse = (img_coarse - 127.5) / 127.5
    img_coarse = np.array(img_coarse)
    mask_coarse = tf.image.resize(mask, (64,64), method=tf.image.ResizeMethod.LANCZOS3)
    mask_coarse = (mask_coarse - 127.5) / 127.5
    mask_coarse = np.array(mask_coarse)
    start_time = time.time()
    X_fakeB_coarse,x_global = g_global_model.predict([img_coarse,mask_coarse])
    
    X_fakeB_coarse = (X_fakeB_coarse + 1) /2.0
    X_fakeB_coarse = cv2.normalize(X_fakeB_coarse, None, alpha = 0, beta = 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    X_fakeB_coarse = tf.image.resize(X_fakeB_coarse, (128,128), method=tf.image.ResizeMethod.LANCZOS3)
    pred_img_coarse = X_fakeB_coarse[:,:,:,0]
    img = (img - 127.5) / 127.5
    mask = (mask - 127.5) / 127.5
    X_fakeB = g_local_model.predict([img,mask,x_global])
    end_time = time.time()
    print(end_time-start_time)
    X_fakeB = (X_fakeB + 1) /2.0
    X_fakeB = cv2.normalize(X_fakeB, None, alpha = 0, beta = 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

    pred_img = X_fakeB[:,:,:,0]
    return np.asarray(pred_img,dtype=np.float32),np.asarray(pred_img_coarse,dtype=np.float32)

In [ ]:
def strided_crop(img, mask, img_h,img_w,height, width,stride=1):
    full_prob = np.zeros((img_h, img_w),dtype=np.float32)
    full_prob_coarse = np.zeros((img_h, img_w),dtype=np.float32)
    full_sum = np.ones((img_h, img_w),dtype=np.float32)
    full_sum_coarse = np.ones((img_h, img_w),dtype=np.float32)
    max_x = int(((img.shape[0]-height)/stride)+1)
    max_y = int(((img.shape[1]-width)/stride)+1)
    max_crops = (max_x)*(max_y)
    i = 0
    for h in range(max_x):
        for w in range(max_y):
                crop_img_arr = img[h * stride:(h * stride) + height,w * stride:(w * stride) + width]
                crop_mask_arr = mask[h * stride:(h * stride) + height,w * stride:(w * stride) + width]
                pred,pred_coarse = normalize_pred(crop_img_arr,crop_mask_arr)
                full_prob[h * stride:(h * stride) + height,w * stride:(w * stride) + width] += pred[0]
                full_sum[h * stride:(h * stride) + height,w * stride:(w * stride) + width] += 1
                full_prob_coarse[h * stride:(h * stride) + height,w * stride:(w * stride) + width] += pred_coarse[0]
                full_sum_coarse[h * stride:(h * stride) + height,w * stride:(w * stride) + width] += 1
                i = i + 1
    out_img = full_prob / full_sum
    out_img_coarse = full_prob_coarse / full_sum_coarse
    return out_img, out_img_coarse

In [ ]:
# Upload h5 models folder
url = "https://drive.google.com/drive/folders/1ubUlq0HIvUMCZyKm-EvcIHBmhlyToOES"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/models/global_model_000020.h5',
 '/content/models/local_model_000020.h5']

In [ ]:
test_data = 'MIC'
out_dir = 'pred'
weight_name_global = 'ICGAN/global_model_000016.h5'
weight_name_local = 'ICGAN/local_model_000016.h5'
stride = 32 # For faster inference use stride 16/32, for better result use stride 3

In [ ]:
## Input dimensions

image_shape_fine = (128,128,3)
mask_shape_fine = (128,128,1)
label_shape_fine = (128,128,1)
image_shape_x_coarse = (64,64,128)
image_shape_coarse = (64,64,3)
mask_shape_coarse = (64,64,1)
label_shape_coarse = (64,64,1)
img_shape_g = (64,64,3)
ndf=64
ncf=128
nff=128

## Load models
K.clear_session()
opt = Adam()
g_local_model = fine_generator(x_coarse_shape=image_shape_x_coarse,input_shape=image_shape_fine,mask_shape=mask_shape_fine,nff=nff)
g_local_model.load_weights(weight_name_local)
g_local_model.compile(loss='mse', optimizer=opt)
g_global_model = coarse_generator(img_shape=image_shape_coarse,mask_shape=mask_shape_coarse,ncf=ncf)
g_global_model.load_weights(weight_name_global)
g_global_model.compile(loss='mse',optimizer=opt)


## Create Output Directory
out_path = out_dir
directories = [out_path,out_path+'/Coarse',out_path+'/Fine']
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

## Find file numbers,paths or names
filenames = glob.glob("MIC/test/images/*.jpg")
limit = len(filenames)

In [ ]:
limit

3

In [ ]:
## Iterating for each image

for i in range(0,limit):

    k = filenames[i].split('/')
    k = k[-1].split('.')[0]
    label_name = "MIC/test/labels/"+k+"_label.png"
    label = Image.open(label_name)
    label_arr = np.asarray(label,dtype=np.float32)
    img_name = "MIC/test/images/"+k+".jpg"
    img = Image.open(img_name)
    img_arr = np.asarray(img,dtype=np.float32)
    mask_name = "MIC/test/mask/"+k+"_mask.png"
    mask = Image.open(mask_name)
    mask_arr = np.asarray(mask,dtype=np.float32)


    ## Get the output predictions as array


    ## Stride =3 (best result),  Stride = 32 (faster result).
    out_img,out_img_coarse = strided_crop(img_arr, mask_arr, mask_arr.shape[0], mask_arr.shape[1], 128, 128, stride)

    out_img[mask_arr==0] = 0
    out_img[out_img>=0.5] = 1
    out_img[out_img<0.5] = 0
    save_im = out_img.astype(np.uint8)
    save_im[save_im==1] = 255
    save_im = Image.fromarray(save_im)


    out_img_coarse[mask_arr==0] = 0
    out_img_coarse[out_img_coarse>=0.5] = 1
    out_img_coarse[out_img_coarse<0.5] = 0
    save_im_coarse = out_img_coarse.astype(np.uint8)
    save_im_coarse[save_im_coarse==1] = 255
    save_im_coarse = Image.fromarray(save_im_coarse)
    
    ## Save files
    pred_name = directories[2]+"/"+k+".png"
    pred_name_coarse = directories[1]+"/"+k+"coarse.png"
    save_im.save(pred_name)
    save_im_coarse.save(pred_name_coarse)
    print("########################### ENDING ITERATION {} ##################" .format(i) )

Streaming output truncated to the last 5000 lines.
0.15162372589111328
1/1 [==============================] - 0s 24ms/step
0.15443944931030273
1/1 [==============================] - 0s 19ms/step
0.15336155891418457
1/1 [==============================] - 0s 20ms/step
0.15499281883239746
1/1 [==============================] - 0s 21ms/step
0.148590087890625
1/1 [==============================] - 0s 19ms/step
0.185164213180542
1/1 [==============================] - 0s 23ms/step
0.1497960090637207
1/1 [==============================] - 0s 21ms/step
0.15036654472351074
1/1 [==============================] - 0s 28ms/step
0.16089534759521484
1/1 [==============================] - 0s 29ms/step
0.15210413932800293
1/1 [==============================] - 0s 20ms/step
0.15667057037353516
1/1 [==============================] - 0s 27ms/step
0.1601104736328125
1/1 [==============================] - 0s 21ms/step
0.16292309761047363
1/1 [==============================] - 0s 22ms/step
0.15468668937683105

## Evaluation

In [ ]:
import glob
import os
import time
import argparse
import numpy as np
from PIL import Image
from libtiff import TIFF
import tensorflow as tf
import cv2
import keras
from keras.optimizers import Adam
from keras.models import Model
import keras.backend as K
from src.model import coarse_generator,fine_generator

# Only this 3 imports are needed additionally for eval
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import jaccard_score
from sklearn.metrics import confusion_matrix,precision_recall_curve,f1_score,roc_auc_score,auc,recall_score, auc,roc_curve


global g_local_model
global g_global_model

In [ ]:
def normalize_pred(img,mask):
    img = np.reshape(img,[1,128,128,3])
    mask = np.reshape(mask,[1,128,128,1])
    img_coarse = tf.image.resize(img, (64,64), method=tf.image.ResizeMethod.LANCZOS3)
    img_coarse = (img_coarse - 127.5) / 127.5
    img_coarse = np.array(img_coarse)
    mask_coarse = tf.image.resize(mask, (64,64), method=tf.image.ResizeMethod.LANCZOS3)
    mask_coarse = (mask_coarse - 127.5) / 127.5
    mask_coarse = np.array(mask_coarse)
    
    _,x_global = g_global_model.predict([img_coarse,mask_coarse])

    img = (img - 127.5) / 127.5
    mask = (mask - 127.5) / 127.5
    X_fakeB = g_local_model.predict([img,mask,x_global])
    X_fakeB = (X_fakeB + 1) /2.0
    X_fakeB = cv2.normalize(X_fakeB, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    pred_img = X_fakeB[:,:,:,0]
    return np.asarray(pred_img,dtype=np.float32)

In [ ]:
def strided_crop(img, mask, img_h,img_w,height, width,stride=1):
  
    full_prob = np.zeros((img_h, img_w),dtype=np.float32)
    full_sum = np.ones((img_h, img_w),dtype=np.float32)
    
    max_x = int(((img.shape[0]-height)/stride)+1)
    #print("max_x:",max_x)
    max_y = int(((img.shape[1]-width)/stride)+1)
    #print("max_y:",max_y)
    max_crops = (max_x)*(max_y)
    i = 0
    for h in range(max_x):
        for w in range(max_y):
                crop_img_arr = img[h * stride:(h * stride) + height,w * stride:(w * stride) + width]
                crop_mask_arr = mask[h * stride:(h * stride) + height,w * stride:(w * stride) + width]
                pred = normalize_pred(crop_img_arr,crop_mask_arr)
                crop_img_arr 
                full_prob[h * stride:(h * stride) + height,w * stride:(w * stride) + width] += pred[0]
                full_sum[h * stride:(h * stride) + height,w * stride:(w * stride) + width] += 1
                i = i + 1
                #print(i)
    out_img = full_prob / full_sum
    return out_img

In [ ]:
test_data = 'CHASE'
# parser.add_argument('--weight_name_global',type=str, help='path/to/global/weight/.h5 file', required=True)
# parser.add_argument('--weight_name_local',type=str, help='path/to/local/weight/.h5 file', required=True)
weight_name_global = 'RVGAN/global_model_000004.h5'
weight_name_local = 'RVGAN/local_model_000004.h5'
stride =32 # For faster inference use stride 16/32, for better result use stride 3

In [ ]:
## Input dimensions

image_shape_fine = (128,128,3)
mask_shape_fine = (128,128,1)
label_shape_fine = (128,128,1)
image_shape_x_coarse = (64,64,128)
image_shape_coarse = (64,64,3)
mask_shape_coarse = (64,64,1)
label_shape_coarse = (64,64,1)
img_shape_g = (64,64,3)
ndf=64
ncf=128
nff=128

## Load models
K.clear_session()
opt = Adam()
g_local_model = fine_generator(x_coarse_shape=image_shape_x_coarse,input_shape=image_shape_fine,mask_shape=mask_shape_fine,nff=nff)
g_local_model.load_weights(weight_name_local)
g_local_model.compile(loss='mse', optimizer=opt)
g_global_model = coarse_generator(img_shape=image_shape_coarse,mask_shape=mask_shape_coarse,ncf=ncf)
g_global_model.load_weights(weight_name_global)
g_global_model.compile(loss='mse',optimizer=opt)


## Find file numbers,paths or names
filenames = glob.glob("CHASE/test/images/*.jpg")
limit = len(filenames)


## Iterating for each image


y_true = np.zeros((limit,960,999),dtype=np.float32)
y_pred = np.zeros((limit,960,999),dtype=np.float32)
y_pred_auc = np.zeros((limit,960,999),dtype=np.float32)
c = 0
for i in range(0,limit):
        
  k = filenames[i].split('/')
  k = k[-1].split('.')[0]
  label_name = "CHASE/test/labels/"+k+"_1stHO.png"
  label = Image.open(label_name)
  label_arr = np.asarray(label,dtype=np.float32)
  img_name = "CHASE/test/images/"+k+".jpg"
  img = Image.open(img_name)
  img_arr = np.asarray(img,dtype=np.float32)
  mask_name = "CHASE/test/mask/"+k+"_mask.png"
  mask = Image.open(mask_name)
  mask_arr = np.asarray(mask,dtype=np.float32)


  ## Get the output predictions as array

  ## Stride =3 (best result),  Stride = 32 (faster result).
  out_img = strided_crop(img_arr, mask_arr, mask_arr.shape[0], mask_arr.shape[1], 128, 128, stride)

  out_img[mask_arr==0] = 0
  y_pred_auc[c,:,:] = out_img
  out_img[out_img>=0.5] = 1
  out_img[out_img<0.5] = 0
  y_true[c,:,:] = label_arr 
  y_pred[c,:,:] = out_img
  c = c +1
    
y_true = y_true.flatten()
y_pred = y_pred.flatten()
y_pred_auc = y_pred_auc.flatten()
confusion = confusion_matrix(y_true, y_pred)
print(confusion)

tn, fp, fn, tp = confusion.ravel()   
metric_cal = time.time()
if float(np.sum(confusion)) != 0:
    accuracy =  float(confusion[0, 0] + confusion[1, 1]) / float(np.sum(confusion))
print("Global Accuracy: " + str(accuracy))
specificity = 0
if float(confusion[0, 0] + confusion[0, 1]) != 0:
    specificity = tn / (tn + fp)
print("Specificity: " + str(specificity))
sensitivity = 0
if float(confusion[1, 1] + confusion[1, 0]) != 0:
    sensitivity = tp / (tp + fn) 
print("Sensitivity: " + str(sensitivity))

precision = 0
if float(confusion[1, 1] + confusion[0, 1]) != 0:
    precision = tp / (tp + fp) 
print("Precision: " + str(precision))


F1_score = 2*tp/(2*tp+fn+fp) 
print("F1 score (F-measure): " + str(F1_score))

AUC_ROC = roc_auc_score(y_true, y_pred_auc)
print("AUC_ROC: " + str(AUC_ROC))

ssim = ssim(y_true, y_pred, data_range=y_true.max()-y_true.min())
print("SSIM: " + str(ssim))

meanIOU = jaccard_score(y_true,y_pred)   # Originaly repo used a fuction called jaccard_similarity_score and had another arguement ,normalize=True
print("meanIOU: " + str(meanIOU))